In [1]:
import numpy as np
import pandas as pd
import plotly.express as px

In this notebook, we will clean the dataset for the *Hot 100 Billboard Charts*. Please make sureto download it from https://www.kaggle.com/datasets/dhruvildave/billboard-the-hot-100-songs and give the correct path to the .csv in the cell below:

In [2]:
data_raw = pd.read_csv("../data/billboardCharts.csv")

In [3]:
data_raw.head(150)

date  rank                      song  \
0    2021-11-06     1                Easy On Me   
1    2021-11-06     2                      Stay   
2    2021-11-06     3             Industry Baby   
3    2021-11-06     4                Fancy Like   
4    2021-11-06     5                Bad Habits   
..          ...   ...                       ...   
145  2021-10-30    46                    Gyalis   
146  2021-10-30    47                 Wild Side   
147  2021-10-30    48                Fair Trade   
148  2021-10-30    49  Leave Before You Love Me   
149  2021-10-30    50                   Deja Vu   

                            artist  last-week  peak-rank  weeks-on-board  
0                            Adele        1.0          1               3  
1    The Kid LAROI & Justin Bieber        2.0          1              16  
2          Lil Nas X & Jack Harlow        3.0          1              14  
3                     Walker Hayes        4.0          3              19  
4                       Ed Sheeran        5.0          2              18  
..                             ...        ...        ...             ...  
145                   Capella Grey       46.0         38              12  
146      Normani Featuring Cardi B       52.0         14              14  
147   Drake Featuring Travis Scott       39.0          3               7  
148    Marshmello X Jonas Brothers       48.0         19              22  
149                 Olivia Rodrigo       42.0          3              29  

[150 rows x 7 columns]

We are only going to look at data from 2020 and 2021, so we can throw away older dates. We are using the same cutoff as the spotify dataset:

In [4]:
data_raw["date"] = pd.to_datetime(data_raw['date'])
truncated = data_raw[(data_raw['date'] > '2019-12-31')]

First, lets see which columns contain missing values:

In [5]:
truncated.isna().sum()

date                 0
rank                 0
song                 0
artist               0
last-week         1536
peak-rank            0
weeks-on-board       0
dtype: int64

In [6]:
data_missing= truncated.isna()
has_nan = data_missing.any(axis=1)
rows_nan = data_missing[has_nan]
indices = rows_nan.index.values
truncated.loc[indices]

date  rank                           song  \
26   2021-11-06    27                Moth To A Flame   
27   2021-11-06    28               Let's Go Brandon   
60   2021-11-06    61                Not In The Mood   
68   2021-11-06    69               Switches & Dracs   
78   2021-11-06    79                    Poke It Out   
...         ...   ...                            ...   
9648 2020-01-04    49  You're A Mean One, Mr. Grinch   
9668 2020-01-04    69       Happy Xmas (War Is Over)   
9684 2020-01-04    85                          Slide   
9697 2020-01-04    98                           Tusa   
9699 2020-01-04   100            South Of The Border   

                                                 artist  last-week  peak-rank  \
26                     Swedish House Mafia & The Weeknd        NaN         27   
27    Bryson Gray Featuring Tyson James & Chandler C...        NaN         28   
60                  Lil Tjay, Fivio Foreign & Kay Flock        NaN         61   
68            Moneybagg Yo Featuring Lil Durk & EST Gee        NaN         69   
78                               Wale Featuring J. Cole        NaN         79   
...                                                 ...        ...        ...   
9648                                  Thurl Ravenscroft        NaN         49   
9668                                        John Legend        NaN         69   
9684                                H.E.R. Featuring YG        NaN         85   
9697                              Karol G & Nicki Minaj        NaN         78   
9699      Ed Sheeran Featuring Camila Cabello & Cardi B        NaN         53   

      weeks-on-board  
26                 1  
27                 1  
60                 1  
68                 1  
78                 1  
...              ...  
9648               1  
9668               1  
9684               1  
9697               4  
9699               3  

[1536 rows x 7 columns]

In [7]:
truncated[truncated["song"].str.contains("Maybach")] #Example for song containing NaN

date  rank     song                    artist  last-week  \
76   2021-11-06    77  Maybach  42 Dugg Featuring Future       85.0   
184  2021-10-30    85  Maybach  42 Dugg Featuring Future       85.0   
284  2021-10-23    85  Maybach  42 Dugg Featuring Future        NaN   
2179 2021-06-12    80  Maybach  42 Dugg Featuring Future       68.0   
2267 2021-06-05    68  Maybach  42 Dugg Featuring Future        NaN   

      peak-rank  weeks-on-board  
76           68               5  
184          68               4  
284          68               3  
2179         68               2  
2267         68               1

As we can see, w have NaN values for the 'last-week' column when the song was not in the charts the previous week. A less error prone way of keeping this information could be to set NaN values to -1:

In [8]:
final = truncated.fillna(-1)

In [9]:
final.isna().sum()

date              0
rank              0
song              0
artist            0
last-week         0
peak-rank         0
weeks-on-board    0
dtype: int64

Empty values are now filled-up, but we observated, that 'last-week' column has float as dtype. The values from this column indicate jumps between ranks, which are always represented as integers. Therefore, we will change the dtype to integer to avoid any kind of problems with future work on our dataset.

In [10]:
# change type of 'last-week' to integer
final = final.astype({'last-week':'int'})
final.dtypes

date              datetime64[ns]
rank                       int64
song                      object
artist                    object
last-week                  int64
peak-rank                  int64
weeks-on-board             int64
dtype: object

The column 'date' is stored as datetime64 with format 'YYYY-MM-DD HH:MM', whereas spotify stores the 'date' in the format 'YYYY-MM-DD'. If we want to compare this dataset with the one of spotify we need to unify the dtypes.

In [11]:
final['date'] = pd.to_datetime(final.date)

# check the type again to see if it is an 'object' now
final.dtypes

date              datetime64[ns]
rank                       int64
song                      object
artist                    object
last-week                  int64
peak-rank                  int64
weeks-on-board             int64
dtype: object

For future work on this dataset, we are exporting the data cleaning to a new csv file.

In [12]:
final.to_csv('../data/billboardCharts_cleaned.csv')